# Getting the most out of NSIDC DAAC data: Harmonizing disparate data for cryosphere remote sensing applications

This tutorial will walk you through data discovery, access, and analysis of NSIDC DAAC data...


Here are the steps you will learn in this tutorial, which will be broken out into several Python Jupyter Notebooks:

Introduction
  
   * Learn the basics of Jupyter Lab and Jupyter Notebook
   * Explore NSIDC data resources
   * Visualize time and area of interest using NASA Worldview / OpenAltimetry 
   
Customize and Access NSIDC DAAC Data   
   
   * Search for data programmatically by time and area of interest.
   * Determine subsetting and reformatting capabilities for our data of interest.
   * Access and customize data using NSIDC's API service.

Analyze 

   * Filter ICESat-2 data by quality flag 
   * Extracting gridded data along ICESat-2 tracks using xarray
   * Plot data with MODIS true color WMS endpoint. 
   * Cartopy is quite good for this.  Basemap is an alternative. OpenAltimetry plotting (3d plotly)



### Basics of Jupyter Lab and Jupyter Notebook

### Import Packages

In [2]:
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import os
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import fiona
import h5py
import re
import import_ipynb
from ipynb.fs.full.Functions import *
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

### Introduction to NSIDC and NSIDC data resources



See the ICESat-2 Data Sets page for a list of all ICESat-2 data set titles and IDs. Below we will input data set ID ATL06, which is the ID for the "ATLAS/ICESat-2 L3A Land Ice Height" data set.

From the ICESat-2 Data Sets page, you can find a link to each data set home page: </br>
ATL03: https://nsidc.org/data/atl03 </br> ATL06: https://nsidc.org/data/atl06 </br> ATL07: https://nsidc.org/data/atl07

From that home page, several resources are available, including an online user guide (within the User Guide tab of the landing page): </br>
ATL03: https://nsidc.org/data/atl03?qt-data_set_tabs=3#qt-data_set_tabs </br> ATL06: https://nsidc.org/data/atl06?qt-data_set_tabs=3#qt-data_set_tabs </br> ATL07: https://nsidc.org/data/atl07?qt-data_set_tabs=3#qt-data_set_tabs

As well as a data dictionary with every data set variable described in detail: </br>
ATL03: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03-data-dictionary-v001.pdf </br> ATL06: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL06-data-dictionary-v001.pdf </br> ATL07: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL07-data-dictionary-v001.pdf

### Visualize time and area of interest



### Sea Ice use case: Explore how ICESat-2 can help map fast ice

OpenAltimetry annotation

https://openaltimetry.org/data/icesat2/?annoId=118&shareKey=5837bf0245ff82d737a1dbd38323f8bd

Worldview link

https://openaltimetry.org/data/icesat2/elevation?minx=133.40944078909814&miny=73.95502623225339&maxx=167.53830223489487&maxy=82.35836552444144&zoom_level=3&beams=1,2,3,4,5,6&tracks=1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306&date=2019-03-23&mapType=arctic

Cycle 2, Track 1298, March 23, 2019. The track crosses the East Siberian Sea, going from land to fast ice, to drift ice including some new ice growth, and is nearly completely clear sky. The thought here is that ICESat-2 could potentially help map fast ice, which is rather difficult to do automatically with satellite imagery. My thought is that the fast ice should have less height/freeboard variability - less surface roughness. It also crosses new ice in a flaw lead as the drift ice moves away from the fast ice edge. So there is a good graduation of ice thickness from new ice to thicker first-year ice. So this could show how well ICESat-2 performs at different ice thicknesses. We could use MODIS reflectance to estimate thin ice thickness, and/or we could also use SMAP. The other neat thing is that there is a cross-over with another track, 1305, on the same day. So could look at the cross-over and compare elevation/freeboard. This point is over drift ice, so it likely wouldn’t be the same floe since it’s 7 orbits later (~12 hours), but if nothing else it would demonstrate how to compare cross-over. (We could use all six beams to get the full 36 crossovers, or maybe just the 3 strong beams to get 9 crossovers - that might be able to capture some of the same floes).

 





### Land Ice use case: Correlate velocity with elevation?

Cycle 1, Track 285, Oct. 17, 2018
Cycle 3, Track 338, Apr. 20, 2019 - might have some clouds
Cycle 2, Track 1283, Mar. 22, 2019
Cycle 2, Track 1291, Mar. 23, 2019 - this crosses the mouth of Jakobshavn, not sure if it would catch the edge of the ice tongue, but may get some sea ice as well



### Data filtering by quality flag




### Xarray to extract gridded data along ICESat-2 tracks


### Plot data with MODIS true color WMS endpoint

In [9]:
url = 'https://gibs.earthdata.nasa.gov/twms/epsg4326/best/twms.cgi?request=GetMap&layers=MODIS_Terra_CorrectedReflectance_TrueColor&srs=EPSG:4326&format=image/jpeg&styles=&time=2012-07-09&width=512&height=512&bbox=-18,27,-13.5,31.5'